# First with the certificate and second without certificate

In [2]:
!pip install paho-mqtt


In [5]:
!pip install git

  Stored in directory: C:\Users\mbene\AppData\Local\pip\Cache\wheels\0b\01\4a\db084059db9f4065752816b5b3b1ba20d8a17a1781432d88ca
  Stored in directory: C:\Users\mbene\AppData\Local\pip\Cache\wheels\da\5f\e1\e059cb94fcd9d85127f5bd5d351e4f5627291845c150e65bf6
Successfully built git pandoc-min


In [8]:
!git clone https://github.com/Azure-Samples/iot-hub-python-raspberrypi-client-app.git

Cloning into 'iot-hub-python-raspberrypi-client-app'...


In [10]:
setup.sh

SyntaxError: invalid syntax (<ipython-input-10-3b8662707c13>, line 1)

In [23]:
from paho.mqtt import client as mqtt
import ssl

path_to_root_cert = "C:\Users\mbene\Documents\Azure Iot\baltimore.cer"
device_id = "PythonDevice"
sas_token = "43Eg9onCO7UWiouyF0X4ay87Tf7NhpCDyDa55MSUMTk="
iot_hub_name = "PruebaIT"

def on_connect(client, userdata, flags, rc):
  print ("Device connected with result code: " + str(rc))
def on_disconnect(client, userdata, rc):
  print ("Device disconnected with result code: " + str(rc))
def on_publish(client, userdata, mid):
  print ("Device sent message")

client = mqtt.Client(client_id=device_id, protocol=mqtt.MQTTv311)

client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_publish = on_publish

client.username_pw_set(username=iot_hub_name+".azure-devices.net/" + device_id, password=sas_token)

client.tls_set(ca_certs=path_to_root_cert, certfile=None, keyfile=None, cert_reqs=ssl.CERT_REQUIRED, tls_version=ssl.PROTOCOL_TLSv1, ciphers=None)
client.tls_insecure_set(False)


Python 

client.connect(iot_hub_name+".azure-devices.net", port=8883)

client.publish("devices/" + device_id + "/messages/events/", "{id=123}", qos=1)
client.loop_forever()


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-23-defaf31bd05b>, line 4)

In [22]:
!pip install azure-iothub-device-client

### Only with the connection string

In [20]:
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project root for full license information.

import random
import time
import sys

# Using the Python Device SDK for IoT Hub:
#   https://github.com/Azure/azure-iot-sdk-python
# The sample connects to a device-specific MQTT endpoint on your IoT Hub.
import iothub_client
# pylint: disable=E0611
from iothub_client import IoTHubClient, IoTHubClientError, IoTHubTransportProvider, IoTHubClientResult
from iothub_client import IoTHubMessage, IoTHubMessageDispositionResult, IoTHubError, DeviceMethodReturnValue

# The device connection string to authenticate the device with your IoT hub.
# Using the Azure CLI:
# az iot hub device-identity show-connection-string --hub-name {YourIoTHubName} --device-id MyNodeDevice --output table
CONNECTION_STRING = "HostName=PruebaIT.azure-devices.net;DeviceId=PythonDevice;SharedAccessKey=43Eg9onCO7UWiouyF0X4ay87Tf7NhpCDyDa55MSUMTk="

# Using the MQTT protocol.
PROTOCOL = IoTHubTransportProvider.MQTT
MESSAGE_TIMEOUT = 10000

# Define the JSON message to send to IoT Hub.
TEMPERATURE = 20.0
HUMIDITY = 60
MSG_TXT = "{\"temperature\": %.2f,\"humidity\": %.2f}"

INTERVAL = 1

def send_confirmation_callback(message, result, user_context):
    print ( "IoT Hub responded to message with status: %s" % (result) )

def iothub_client_init():
    # Create an IoT Hub client
    client = IoTHubClient(CONNECTION_STRING, PROTOCOL)
    return client

# Handle direct method calls from IoT Hub
def device_method_callback(method_name, payload, user_context):
    global INTERVAL
    print ( "\nMethod callback called with:\nmethodName = %s\npayload = %s" % (method_name, payload) )
    device_method_return_value = DeviceMethodReturnValue()
    if method_name == "SetTelemetryInterval":
        try:
            INTERVAL = int(payload)
            # Build and send the acknowledgment.
            device_method_return_value.response = "{ \"Response\": \"Executed direct method %s\" }" % method_name
            device_method_return_value.status = 200
        except ValueError:
            # Build and send an error response.
            device_method_return_value.response = "{ \"Response\": \"Invalid parameter\" }"
            device_method_return_value.status = 400
    else:
        # Build and send an error response.
        device_method_return_value.response = "{ \"Response\": \"Direct method not defined: %s\" }" % method_name
        device_method_return_value.status = 404
    return device_method_return_value

def iothub_client_telemetry_sample_run():

    try:
        client = iothub_client_init()
        print ( "IoT Hub device sending periodic messages, press Ctrl-C to exit" )

        # Set up the callback method for direct method calls from the hub.
        client.set_device_method_callback(
            device_method_callback, None)

        while True:
            # Build the message with simulated telemetry values.
            temperature = TEMPERATURE + (random.random() * 15)
            humidity = HUMIDITY + (random.random() * 20)
            msg_txt_formatted = MSG_TXT % (temperature, humidity)
            message = IoTHubMessage(msg_txt_formatted)

            # Add a custom application property to the message.
            # An IoT hub can filter on these properties without access to the message body.
            prop_map = message.properties()
            if temperature > 30:
              prop_map.add("temperatureAlert", "true")
            else:
              prop_map.add("temperatureAlert", "false")

            # Send the message.
            print( "Sending message: %s" % message.get_string() )
            client.send_event_async(message, send_confirmation_callback, None)
            time.sleep(INTERVAL)

    except IoTHubError as iothub_error:
        print ( "Unexpected error %s from IoTHub" % iothub_error )
        return
    except KeyboardInterrupt:
        print ( "IoTHubClient sample stopped" )

if __name__ == '__main__':
    print ( "IoT Hub Quickstart #2 - Simulated device" )
    print ( "Press Ctrl-C to exit" )
    iothub_client_telemetry_sample_run()

IoT Hub Quickstart #2 - Simulated device
Press Ctrl-C to exit
IoT Hub device sending periodic messages, press Ctrl-C to exit
Sending message: {"temperature": 26.87,"humidity": 60.07}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 24.14,"humidity": 60.24}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 21.97,"humidity": 74.49}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 24.12,"humidity": 77.24}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 28.85,"humidity": 69.13}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 30.79,"humidity": 67.15}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 25.81,"humidity": 80.00}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 33.97,"humidity": 72.50}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 23.64,"humidity": 

In [21]:
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project root for full license information.

import random
import time
import sys

# Using the Python Device SDK for IoT Hub:
#   https://github.com/Azure/azure-iot-sdk-python
# The sample connects to a device-specific MQTT endpoint on your IoT Hub.
import iothub_client
# pylint: disable=E0611
from iothub_client import IoTHubClient, IoTHubClientError, IoTHubTransportProvider, IoTHubClientResult
from iothub_client import IoTHubMessage, IoTHubMessageDispositionResult, IoTHubError, DeviceMethodReturnValue

# The device connection string to authenticate the device with your IoT hub.
# Using the Azure CLI:
# az iot hub device-identity show-connection-string --hub-name {YourIoTHubName} --device-id MyNodeDevice --output table
CONNECTION_STRING = "HostName=PruebaIT.azure-devices.net;DeviceId=PythonDevice;SharedAccessKey=43Eg9onCO7UWiouyF0X4ay87Tf7NhpCDyDa55MSUMTk="

# Using the MQTT protocol.
PROTOCOL = IoTHubTransportProvider.MQTT
MESSAGE_TIMEOUT = 10000

# Define the JSON message to send to IoT Hub.
TEMPERATURE = 20.0
HUMIDITY = 60
MSG_TXT = "{\"temperature\": %.2f,\"humidity\": %.2f}"

def send_confirmation_callback(message, result, user_context):
    print ( "IoT Hub responded to message with status: %s" % (result) )

def iothub_client_init():
    # Create an IoT Hub client
    client = IoTHubClient(CONNECTION_STRING, PROTOCOL)
    return client

def iothub_client_telemetry_sample_run():

    try:
        client = iothub_client_init()
        print ( "IoT Hub device sending periodic messages, press Ctrl-C to exit" )

        while True:
            # Build the message with simulated telemetry values.
            temperature = TEMPERATURE + (random.random() * 15)
            humidity = HUMIDITY + (random.random() * 20)
            msg_txt_formatted = MSG_TXT % (temperature, humidity)
            message = IoTHubMessage(msg_txt_formatted)

            # Add a custom application property to the message.
            # An IoT hub can filter on these properties without access to the message body.
            prop_map = message.properties()
            if temperature > 30:
              prop_map.add("temperatureAlert", "true")
            else:
              prop_map.add("temperatureAlert", "false")

            # Send the message.
            print( "Sending message: %s" % message.get_string() )
            client.send_event_async(message, send_confirmation_callback, None)
            time.sleep(1)

    except IoTHubError as iothub_error:
        print ( "Unexpected error %s from IoTHub" % iothub_error )
        return
    except KeyboardInterrupt:
        print ( "IoTHubClient sample stopped" )

if __name__ == '__main__':
    print ( "IoT Hub Quickstart #1 - Simulated device" )
    print ( "Press Ctrl-C to exit" )
    iothub_client_telemetry_sample_run()

IoT Hub Quickstart #1 - Simulated device
Press Ctrl-C to exit
IoT Hub device sending periodic messages, press Ctrl-C to exit
Sending message: {"temperature": 20.14,"humidity": 78.93}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 22.07,"humidity": 60.35}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 23.38,"humidity": 65.93}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 22.80,"humidity": 61.17}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 27.27,"humidity": 76.60}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 31.95,"humidity": 61.44}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 29.55,"humidity": 73.40}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 29.37,"humidity": 66.39}
IoT Hub responded to message with status: OK
Sending message: {"temperature": 33.72,"humidity": 

## Device for pruevaanalytics